****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize

## Read json file

In [2]:
json_dict = json.load((open('data/world_bank_projects.json')))

Taking a look at keys from the first dictionary

In [3]:
json_dict[0].keys()

dict_keys(['mjtheme_namecode', 'majorsector_percent', 'lendinginstrtype', 'theme1', 'regionname', 'country_namecode', 'sector4', 'prodlinetext', 'themecode', 'countrycode', 'sector', 'countryname', 'url', 'prodline', 'projectstatusdisplay', 'sector3', 'idacommamt', 'status', 'ibrdcommamt', 'impagency', 'lendprojectcost', 'mjsector_namecode', 'grantamt', 'project_abstract', 'totalcommamt', 'lendinginstr', 'theme_namecode', 'sector_namecode', 'sector2', 'projectdocs', 'id', 'sector1', 'mjtheme', 'mjthemecode', 'supplementprojectflg', 'productlinetype', 'countryshortname', '_id', 'project_name', 'board_approval_month', 'boardapprovaldate', 'approvalfy', 'envassesmentcategorycode', 'borrower', 'projectfinancialtype', 'docty', 'source', 'closingdate', 'totalamt', 'sectorcode'])

In [4]:
json_dict[0].get('countryname')

'Federal Democratic Republic of Ethiopia'

## Create pandas dataframe from list of dict

In [5]:
# create Dataframe from json dict
df = json_normalize(json_dict)
# df.info()
# df.head()

## Answer to question 1
Each entry represents a project. A simple value_counts() will return top contributors in terms of project counts.<br>
PS: I thought Africa was a continent.

### Solution A

In [6]:
df.countryname.value_counts()[:10]

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

### Solution B
More complicated than it needs to be

In [7]:
# group by country and count on aggregate, sort in descending order
s_proj = df.groupby('countryname')['project_name'].count().sort_values(ascending = False)
# show the first 10
s_proj[:10]

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: project_name, dtype: int64

## Answer to question 2
Top 10 major project themes
In `'mjtheme_namecode'` there are pairs (code,name). json_normalize() unstack those in dataframe.

In [8]:
# only keep content of mjtheme_namecode
df2 = json_normalize(json_dict, 'mjtheme_namecode')
# df2.head()

In [9]:
df2.name.value_counts()[:10]

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

## Answer to question 3
Fill up missing themes. Just verifying there is a unique name for each code (apart from empty string).

In [10]:
df2.groupby('code')['name'].value_counts()

code  name                                        
1     Economic management                              33
                                                        5
10    Rural development                               202
                                                       14
11    Environment and natural resources management    223
                                                       27
2     Public sector governance                        184
                                                       15
3     Rule of law                                      12
                                                        3
4     Financial and private sector development        130
                                                       16
5     Trade and integration                            72
                                                        5
6     Social protection and risk management           158
                                                       10
7     Social dev/gend

### Solution A

In [11]:
# criteria for missing value
mask_empty = df2.name == ''

# discard missing values
df2_unique = df2[~mask_empty]

# drop duplicates
df2_unique = df2_unique.drop_duplicates()

# set codes as index
df2_unique = df2_unique.set_index('code')

# fill in missing values
df2.loc[mask_empty, 'name'] = df2.loc[mask_empty, 'code'].map(df2_unique.name)

In [12]:
df2.name.value_counts()[:10]

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64

### Solution B

In [13]:
# criteria for missing value
mask_empty = df2.name == ''

# discard missing values
df2_unique = df2[~mask_empty]

# determine most frequently occurring theme for each code
s2_unique = df2_unique.groupby('code')['name'].apply(lambda r: r.mode())

# drop 2nd level multi-index created by mode(), there is only one name for each code
s2_unique.index = s2_unique.index.droplevel(1)

# create a ndarray for each code index in df2 based on s2_unique
code_col = s2_unique.reindex(df2.code.values).values

# assign missing values
df2.loc[mask_empty, 'name'] = code_col[mask_empty]

In [14]:
df2.name.value_counts()[:10]

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64